**Simple example for Asg1 with handcrafted feature**

An example diagram for instance search.
Please read the slide "Information for Asg1" first.

In every bounding box file (the txt files), each line records the coordinates of one bounding box in format: x of top-left point, y of top-left point, width, height.

The bounding box information is only provided for the query images.

Notes: it's possible that there are more than one instances in a query image (see the annotation(s) in the corresponding txt file), you should find the most similar image rank list in the gallery by considering all the instances.

In [226]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 
import matplotlib.pyplot as plt
from tqdm import tqdm
import threading


 Download the dataset from this linke [onedrive link](https://portland-my.sharepoint.com/:u:/g/personal/srwang3-c_my_cityu_edu_hk/EZ0BIZatMIJMoiCG4-uy6okBSlXLJD7TUyIDG1lbKUJ0eA?e=lbKz4a).
And unzip the downloaded file into some path. 
> In this tutorial, the path is `/Users/txsing/datasets_4186`, please replace to your own accordingly.

Initialize the necessary parameters, including paths, feature extractors.

In [227]:
download_path='D:/Users/KLIVANCHAN/Desktop/CityU' # change to your own download path
path_query=download_path+'/query_4186'
path_query_txt=download_path+'/query_txt_4186'

# path_query_txt is the directory to the bounding box information of the instance(s) for the query images
path_gallery=download_path+'/gallery_4186'

name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)

sift = cv2.ORB_create()
# bf = cv2.BFMatcher()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
# search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,{})

record_all=np.zeros((num_query,len(name_gallery)))

query_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_query+'/*.jpg')]
query_imgs_no = [x[:-4] for x in query_imgs_no]

gallery_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_gallery+'/*.jpg')]
gallery_imgs_no = [x[:-4] for x in gallery_imgs_no]

Process

Feature extraction -> calculate distance -> compare

In [228]:
def instance_search(num):
    for i, query_img_no in enumerate(query_imgs_no[num]):
    
    time_s = time.time()
    dist_record=[]
    per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
    per_query=cv2.imread(per_query_name)
    gallery_imgs_no_desc=[]
    
    # read boundary from text file
    queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
    boundary_file = open(queryfilename, 'r')
    boundary = boundary_file.readline().strip().split(' ')
    boundary = [int(b) for b in boundary]
    boundary_file.close()
    
    # crop the image
    x ,y, w, h = boundary
    query_boundary = per_query[y:y+h, x:x+w]
    
    # feature extraction for per query
    # the bounding box information is not considered
    # quite naive, just an example
    per_query_kp, per_query_des = sift.detectAndCompute(query_boundary,None)
    
    # the iteration loop for gallery
    for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no), desc=f"Processing query part {i}"):
        per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
        per_gallery=cv2.imread(per_gallery_name)
        
        # feature extraction for per gallery
        per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery,None)
        
        # use part of the features to make the calculation feasible
        # quite naive, just an example
#         matches = bf.knnMatch(np.asarray(per_query_des,np.float32),np.asarray(per_gallery_des,np.float32),k=2)
        matches = flann.knnMatch(per_query_des.astype(np.float32),per_gallery_des.astype(np.float32),k=2)
    
        # Compute similarity score for each match
        sim_scores = []
        for match in matches:
            if len(match) == 2:
                (m,n) = match
                query_desc = per_query_des[m.queryIdx]
                gallery_desc = per_gallery_des[m.trainIdx]
                sim_score = np.sum((np.double(query_desc)-np.double(gallery_desc))**2)/np.prod(np.shape(query_desc))
                if np.isnan(sim_score):  # Check if similarity score is NaN
                    continue
                sim_scores.append(sim_score)

        # Compute average similarity score for all matches
        avg_sim_score = np.mean(sim_scores)
        dist_record.append(avg_sim_score)
#         cv2.imshow('result',per_result_img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

    # find the indexes with descending similarity order
    ascend_index=sorted(range(len(dist_record)), key=lambda k: dist_record[k])
    # update the results for one query
    for k in range(len(ascend_index)):
        gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[ascend_index[k]])
    record_all[i,:]= gallery_imgs_no_desc
    time_e = time.time()
    sorted(dist_record)
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
    query_idx = i
    print(f'For query image No. {query_imgs_no[query_idx]}, the top ranked similar image No. is {gallery_imgs_no_desc[0]}.')
    
 
 
if __name__ =="__main__":
    # creating thread
    threadList = ["Thread-1", "Thread-2", "Thread-3"]
    threadID = 1
    for tName in threadList:
       thread = threading.Thread(target=instance_search, args=(i,))
       thread.start()
       threads.append(thread)
       threadID += 1

    for t in threads:
        t.join()
    print("Done!")



# the iteration loop for query 
# iteration times is 2 here, only for the demonstration
for i, query_img_no in enumerate(query_imgs_no[0:20]):
    
    time_s = time.time()
    dist_record=[]
    per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
    per_query=cv2.imread(per_query_name)
    gallery_imgs_no_desc=[]
    
    # read boundary from text file
    queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
    boundary_file = open(queryfilename, 'r')
    boundary = boundary_file.readline().strip().split(' ')
    boundary = [int(b) for b in boundary]
    boundary_file.close()
    
    # crop the image
    x ,y, w, h = boundary
    query_boundary = per_query[y:y+h, x:x+w]
    
    # feature extraction for per query
    # the bounding box information is not considered
    # quite naive, just an example
    per_query_kp, per_query_des = sift.detectAndCompute(query_boundary,None)
    
    # the iteration loop for gallery
    for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no), desc=f"Processing query part {i}"):
        per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
        per_gallery=cv2.imread(per_gallery_name)
        
        # feature extraction for per gallery
        per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery,None)
        
        # use part of the features to make the calculation feasible
        # quite naive, just an example
#         matches = bf.knnMatch(np.asarray(per_query_des,np.float32),np.asarray(per_gallery_des,np.float32),k=2)
        matches = flann.knnMatch(per_query_des.astype(np.float32),per_gallery_des.astype(np.float32),k=2)
    
        # Compute similarity score for each match
        sim_scores = []
        for match in matches:
            if len(match) == 2:
                (m,n) = match
                query_desc = per_query_des[m.queryIdx]
                gallery_desc = per_gallery_des[m.trainIdx]
                sim_score = np.sum((np.double(query_desc)-np.double(gallery_desc))**2)/np.prod(np.shape(query_desc))
                if np.isnan(sim_score):  # Check if similarity score is NaN
                    continue
                sim_scores.append(sim_score)

        # Compute average similarity score for all matches
        avg_sim_score = np.mean(sim_scores)
        dist_record.append(avg_sim_score)
#         cv2.imshow('result',per_result_img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

    # find the indexes with descending similarity order
    ascend_index=sorted(range(len(dist_record)), key=lambda k: dist_record[k])
    # update the results for one query
    for k in range(len(ascend_index)):
        gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[ascend_index[k]])
    record_all[i,:]= gallery_imgs_no_desc
    time_e = time.time()
    sorted(dist_record)
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
    query_idx = i
    print(f'For query image No. {query_imgs_no[query_idx]}, the top ranked similar image No. is {gallery_imgs_no_desc[0]}.')

Processing query part 0: 5000it [02:34, 32.32it/s]


retrieval time for query 1258 is 158.28146815299988s
For query image No. 1258, the top ranked similar image No. is 2403.


Processing query part 1: 2511it [01:20, 31.20it/s]


KeyboardInterrupt: 

Output 

Save the rank list file.

In [ ]:
# write the output file following the example
f=open(r'./rank_list.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()